In [8]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import sqlite3
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [27]:
conn = sqlite3.connect('./dealerships_scraper/data/cars_test.db')
#conn = sqlite3.connect('demo.db')

sql_query = '''

      SELECT
          count(*) 
      FROM
        inventory_test;

'''
result = pd.read_sql_query(sql_query, conn)
result[:3]

,count(*)
0,187


In [26]:
# WBA73BJ04MWX00541
conn = sqlite3.connect('./dealerships_scraper/data/cars_test.db')
sql_query = '''

      SELECT
          * 
      FROM
        inventory_test where vin = 'WBA73BJ04MWX00541';

'''
result = pd.read_sql_query(sql_query, conn)
result

,vin,title,year,make,model,trim,model_trim,price,mileage,vehicle_type,interior_color,exterior_color,transmission,engine,drivetrain
0,WBA73BJ04MWX00541,2021 BMW 5 Series 540i xDrive Sedan,2021,BMW,5 Series,540i xDrive,BMW 5 Series,57990.0,20966,Sedan,Black,Bernina Grey Amber Effect Metallic,Automatic,Intercooled Turbo Gas/Electric I-6 3.0 L/183,AWD


In [23]:
####### Drop Table

TABLE_NAME = 'inventory_test'
DB_NAME = './dealerships_scraper/data/cars_test.db'

#Connecting to sqlite
conn = sqlite3.connect(DB_NAME)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute(f"DROP TABLE {TABLE_NAME}")
print("Table dropped... ")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()


Table dropped... 
